In [1]:
from scipy.io import loadmat
import numpy as np
import torch
import math

Let $r_i$ and $I_i$ be the population-firing rate and total synaptic input current for population $i$, the firing rate is determined by:
$$
r_i=F(I_i)=\cfrac{aI_i-b}{1-\exp{(-d(aI_i-b))}},
$$
where $a=270{\rm Hz/nA}$, $b=108{\rm Hz}$, $d=0.154{\rm sec}$.

The input current $I_i$ is given by:
$$
I_i = w J_N S_i + G J_N \sum ^N_{j=1} C_{ij} S_j + I_{bi},
$$
where $J_N$ is the overall excitatory strength, $w$ and $G$ scale the strengths of local and long-range interactions, and $S_i$ is the synaptic drive variable.

$I_{bi}$ is the background input into population $i$ follows:
$$
\tau_0\cfrac{dI_{bi}}{dt}=-(I_{bi}-I_0)+\eta_i(t)\sqrt{\tau_0\sigma^2},
$$
where $I_0$ is the mean of $I_{bi}$, $\tau_0$ is the filter time constant, $\sigma$ is the noise amplitude, $\eta(t)$ is a Gaussian white-noise with zero mean and unit standard deviation.

The synaptic drive variable $S_i$ for population $i$ obeys:
$$
\cfrac{dS_i}{dt}=F(I_i)\gamma(1-S_i)-\cfrac{1}{\tau_s}S_i,
$$
where $\tau_s$ and $\gamma$ are constants.

The BOLD signal $B(t)$ generated by $S_i(t)$ is computed by evaluating the causal convolution of $S_i(t)$ with HRF kernel $f_{bold}(t)$:
$$
B_i(t)=\int^t_{-\infty}S_i(x)f_{bold}(t-x)dx.
$$

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def F(I, a = 270., b = 108., d = 0.154):

    "Transfer function used to determine population-firing rate r with synaptic input I"
    
    return (a*I-b) / (1.-np.exp(-d*(a*I-b)))



def fbold(t, p = 2, taubold = 1.25, o = 2.25):
    
    "Boynton gamma function used as a HRF kernel"

    kernel = np.zeros_like(t)
    idx = (t >= o)
    kernel[idx] = ((t[idx]-o)/taubold)**(p-1) / math.factorial(p-1) * np.exp(-(t[idx]-o)/taubold)
    
    return kernel

In [4]:
load_data = loadmat('./Human_66.mat')
C = np.array(load_data['C'])
anat_labels = load_data['anat_lbls']
order = np.array(load_data['Order'][0]) - 1
C = (C[order].T)[order].T
anat_labels = anat_labels[order]

In [5]:
C = C                   # structural connectivity
N = C.shape[0]          # number of nodes
w = 0.55                # self-excitation scaling factor
G = 3.5                 # long-range scaling factor
JN = 0.2609             # overall excitatory strength
I0 = 0.3255             # mean of background input
tau0 = 0.002            # noise time constant
sigma = 0.005           # noise amplitude
tauS = 0.1              # synaptic time constant
gamma = 0.641           # saturation factor for gating variable
dt = 0.002              # dt of Newton's method

In [6]:
def data_simulation(T, S_start, Ib_start, Ib_in, start, step):

    range_t = np.arange(0, T, dt); nt = range_t.size

    S = np.zeros((nt, N)); S[0] = S_start
    r = np.zeros((nt, N))
    B = torch.zeros(nt, N).type(torch.float32).to(device)
    IB = np.zeros((nt, N))

    fbold_vec = torch.from_numpy(fbold(range_t)[:int(60/dt)]).type(torch.float32)
    reversed_fbold_vec = fbold_vec.flip(0).to(device)

    I = np.zeros(N); Ib = Ib_start; IB[0] = Ib

    for idx in range(1, nt):

        I = w * JN * S[idx-1] + G * JN * C @ S[idx-1] + Ib
        r[idx] = F(I)
        S[idx] = S[idx-1] + dt * (gamma * r[idx] * (1 - S[idx-1]) - S[idx-1] / tauS)
        Ib = Ib + (dt / tau0) * (-(Ib - I0) + np.sqrt(tau0) * sigma * Ib_in[idx])
        IB[idx] = Ib

    S_res = S[start::step]
    Ib_res = IB[start::step]
    S = torch.from_numpy(S).type(torch.float32).to(device)

    for idx in range(N):
        
        padding_input = torch.nn.functional.pad(S[:, idx], (fbold_vec.size(0)-1, 0), mode = 'constant', value = 0)
        B[:, idx] = torch.nn.functional.conv1d(padding_input.view(1, 1, -1), reversed_fbold_vec.view(1, 1, -1))[0][0]
    
    B_res = B[start::step].detach().cpu().numpy()
    
    del(S); del(B)

    return S_res, Ib_res, B_res

In [7]:
for random_seed in range(50):

    TR = 0.72; step = int(TR / dt); start = step - 1
    T = int(TR * 12000)
    S_start = np.zeros(N); r_start = np.zeros(N)
    np.random.seed(random_seed)
    Ib_in = np.random.randn(int(T/dt), N)
    Ib_start = sigma * Ib_in[0]
    S_res, Ib_res, B_res = data_simulation(T, S_start, Ib_start, Ib_in, start, step)
    np.savetxt('./sim_data/dynamics_raw_T_' + str(T) + '_TR_' + str(int(1000 * TR)) + '_seed_' + '{:03d}'.format(random_seed) + '.txt', B_res)
    
    pert_step = 500
    pert_T = 5
    pert_start_list = list(range(2000, B_res.shape[0]-int(pert_T/TR), pert_step))
    realEC = np.zeros((N, N))
    for pert_start in pert_start_list:
        S_start = S_res[pert_start-1]; Ib_start = Ib_res[pert_start-1]
        Ib_in_unpert = Ib_in[start+pert_start*step:start+pert_start*step+int(pert_T/dt), :]
        _, _, B_unpert = data_simulation(pert_T, S_start, Ib_start, Ib_in_unpert, start, step)
        unpert = B_unpert[4]
        for i in range(N):
            Ib_in_pert = Ib_in_unpert.copy()
            perturb = np.zeros(N); perturb[i] = 5.0
            Ib_in_pert[:int(0.1/dt)] = Ib_in_pert[:int(0.1/dt)] + perturb
            _, _, B_pert = data_simulation(pert_T, S_start, Ib_start, Ib_in_pert, start, step)
            realEC[i] = realEC[i] + (B_pert[4] - unpert)
    realEC = realEC / len(pert_start_list)
    np.savetxt('./sim_data/real_EC_seed_' + '{:03d}'.format(random_seed) + '.txt', realEC)